# AWS Machine Learning Nandoegree Capstone Project
# Forecasting with Amazon Forecast

## Note! These steps were taken from the below reference Forecast walkthrough: 
https://github.com/aws-samples/amazon-forecast-samples/blob/main/notebooks/basic/Getting_Started/Amazon_Forecast_Quick_Start_Guide.ipynb
https://github.com/aws-samples/amazon-forecast-samples/blob/main/notebooks/common/util/fcst_utils.py

#### Setup Notebook Environment

In [1]:
%%capture --no-stderr setup

!pip install pandas s3fs matplotlib ipywidgets
!pip install boto3 --upgrade

%reload_ext autoreload

#### Setup Imports

In [19]:
import sys
import os
import glob 
#sys.path.insert( 0, os.path.abspath("../../common") )

import json
from util import * #.fcst_utils import *
import boto3
import s3fs
import pandas as pd

#### Setup IAM Role used by Amazon Forecast to access your data

In [2]:
#role was manually setup in AWS console, with AmazonS3FullAccess
role_arn = 'arn:aws:iam::054619787751:role/my-forecast-role'

#### Create an instance of AWS SDK client for Amazon Forecast

In [3]:
region = 'us-east-1'
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

# Checking to make sure we can communicate with Amazon Forecast
assert forecast.list_predictors()

## Step 1: Import your data. <a class="anchor" id="import"></a>

In this step, we will create a **Dataset** and **Import** the Taiwan stock dataset from S3 to Amazon Forecast. To train a Predictor we will need a **DatasetGroup** that groups the input **Datasets**. So, we will end this step by creating a **DatasetGroup** with the imported **Dataset**.

In [4]:
s3 = boto3.Session().resource('s3')
bucket_name = "forecast-exp-1111"

In [6]:
keys=[]
files = glob.glob(os.path.join(os.getcwd(), "forecast_import", "*"))
for file in files:
    keys.append(r"forecast_import/"+os.path.split(file)[1])

In [7]:
keys

['forecast_import/target_wl.parquet']

In [16]:
for key in keys:
    s3.Bucket(bucket_name).Object(key).upload_file(key)
    ts_s3_path = f"s3://{bucket_name}/{key}"

print(f"\nDone, the dataset is uploaded to S3 at {ts_s3_path}.")


Done, the dataset is uploaded to S3 at s3://forecast-exp-1111/forecast_import/target_wl.parquet.


#### Creating the Dataset

In [ ]:
#ONLY NEED TO RUN THIS ONCE. SKIP TO NEXT CELL IF CREATED PREVIOUSLY.
DATASET_FREQUENCY = "D" # H for hourly.
TS_DATASET_NAME = "WATCHLIST_TS"
TS_SCHEMA = {
   "Attributes":[
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      
      {
         "AttributeName":"target_value",
         "AttributeType":"integer"
      }
   ]
}

create_dataset_response = forecast.create_dataset(Domain="CUSTOM",
                                                  DatasetType='TARGET_TIME_SERIES',
                                                  DatasetName=TS_DATASET_NAME,
                                                  DataFrequency=DATASET_FREQUENCY,
                                                  Schema=TS_SCHEMA)

ts_dataset_arn = create_dataset_response['DatasetArn']
describe_dataset_response = forecast.describe_dataset(DatasetArn=ts_dataset_arn)

print(f"The Dataset with ARN {ts_dataset_arn} is now {describe_dataset_response['Status']}")

In [6]:
#ONLY RUN IF YOU HAVE ALREADY RUN THE ABOVE CELL
#Obtained arn from error message when running above cell after already created
ts_dataset_arn = 'arn:aws:forecast:us-east-1:054619787751:dataset/WATCHLIST_TS'

#### Importing the Dataset

In [ ]:
#ONLY NEED TO RUN THIS ONCE. SKIP TO NEXT CELL IF CREATED PREVIOUSLY.
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"
TS_IMPORT_JOB_NAME = "PREFUNDING_TTS_IMPORT"
TIMEZONE = "EST"

ts_dataset_import_job_response = \
    forecast.create_dataset_import_job(DatasetImportJobName=TS_IMPORT_JOB_NAME,
                                       DatasetArn=ts_dataset_arn,
                                       DataSource= {
                                         "S3Config" : {
                                             "Path": ts_s3_path,
                                             "RoleArn": role_arn
                                         } 
                                       },
                                       Format="PARQUET",
                                       TimestampFormat=TIMESTAMP_FORMAT,
                                       TimeZone = TIMEZONE)

ts_dataset_import_job_arn = ts_dataset_import_job_response['DatasetImportJobArn']
describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)

print(f"Waiting for Dataset Import Job with ARN {ts_dataset_import_job_arn} to become ACTIVE. This process could take 5-10 minutes.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))

describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn)
print(f"\n\nThe Dataset Import Job with ARN {ts_dataset_import_job_arn} is now {describe_dataset_import_job_response['Status']}.")

Waiting for Dataset Import Job with ARN arn:aws:forecast:us-east-1:054619787751:dataset-import-job/WATCHLIST_TS/PREFUNDING_TTS_IMPORT to become ACTIVE. This process could take 5-10 minutes.

Current Status:
CREATE_PENDING .
CREATE_IN_PROGRESS ............................................................

In [8]:
#ONLY RUN IF YOU HAVE ALREADY RUN THE ABOVE CELL
# target dataset (watchlist) imported at:
ts_dataset_import_job_arn = 'arn:aws:forecast:us-east-1:054619787751:dataset-import-job/WATCHLIST_TS/PREFUNDING_TTS_IMPORT'

#### Creating a DatasetGroup

In [99]:
#ONLY NEED TO RUN THIS ONCE. SKIP TO NEXT CELL IF CREATED PREVIOUSLY.
DATASET_GROUP_NAME = "TAIWAN_PREFUNDING"
DATASET_ARNS = [ts_dataset_arn]

create_dataset_group_response = \
    forecast.create_dataset_group(Domain="CUSTOM",
                                  DatasetGroupName=DATASET_GROUP_NAME,
                                  DatasetArns=DATASET_ARNS)

dataset_group_arn = create_dataset_group_response['DatasetGroupArn']
describe_dataset_group_response = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

print(f"The DatasetGroup with ARN {dataset_group_arn} is now {describe_dataset_group_response['Status']}.")

The DatasetGroup with ARN arn:aws:forecast:us-east-1:054619787751:dataset-group/TAIWAN_PREFUNDING is now ACTIVE.


In [9]:
#ONLY RUN IF YOU HAVE ALREADY RUN THE ABOVE CELL
#Obtained arn original cell execution above
dataset_group_arn = 'arn:aws:forecast:us-east-1:054619787751:dataset-group/TAIWAN_PREFUNDING'

## Step 2: Train a predictor - Experiment 01 <a class="anchor" id="predictor"></a>

In this step, we will create a **Predictor** using the **DatasetGroup** that was created above. After creating the predictor, we will review the accuracy obtained through the backtesting process to get a quantitative understanding of the performance of the predictor.

This will be the baseline predictor and experiment which we will expand on later with related datasets.

In [ ]:
#ONLY NEED TO RUN THIS ONCE. SKIP TO NEXT CELL IF CREATED PREVIOUSLY.
PREDICTOR_NAME = "PREFUNDING_PREDICTOR_01"
FORECAST_HORIZON = 1
FORECAST_FREQUENCY = "D"
#HOLIDAY_DATASET = [{
#        'Name': 'holiday',
#        'Configuration': {
#        'CountryCode': ['TW']
#    }
#}]

create_auto_predictor_response = \
    forecast.create_auto_predictor(PredictorName = PREDICTOR_NAME,
                                   ForecastHorizon = FORECAST_HORIZON,
                                   ForecastFrequency = FORECAST_FREQUENCY,
                                   DataConfig = {
                                       'DatasetGroupArn': dataset_group_arn
                                       #,'AdditionalDatasets': HOLIDAY_DATASET
                                        },
                                   ExplainPredictor = True)

predictor_arn = create_auto_predictor_response['PredictorArn']
print(f"Waiting for Predictor with ARN {predictor_arn} to become ACTIVE. Depending on data size and predictor setting，it can take several hours to be ACTIVE.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=predictor_arn))

describe_auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=predictor_arn)
print(f"\n\nThe Predictor with ARN {predictor_arn} is now {describe_auto_predictor_response['Status']}.")

Waiting for Predictor with ARN arn:aws:forecast:us-east-1:054619787751:predictor/PREFUNDING_PREDICTOR_01_01GKW0Q4KG85PR41MRZXVXR7F5 to become ACTIVE. Depending on data size and predictor setting，it can take several hours to be ACTIVE.

Current Status:
CREATE_PENDING ..
CREATE_IN_PROGRESS .................

In [5]:
#ONLY RUN IF YOU HAVE ALREADY RUN THE ABOVE CELL
#Obtained arn original cell execution above
predictor_arn = 'arn:aws:forecast:us-east-1:054619787751:predictor/PREFUNDING_PREDICTOR_01_01GKW0Q4KG85PR41MRZXVXR7F5'

#### Review accuracy metrics

* **Weighted Quantile Loss (wQL)** metric measures the accuracy of a model at a specified quantile. It is particularly useful when there are different costs for underpredicting and overpredicting.

* **Root Mean Square Error (RMSE)** uses the squared value of the residuals, which amplifies the impact of outliers. In use cases where only a few large mispredictions can be very costly, the RMSE is the more relevant metric.

* **Weighted Absolute Percentage Error (WAPE)** is more robust to outliers than Root Mean Square Error (RMSE) because it uses the absolute error instead of the squared error.

* **Mean Absolute Percentage Error (MAPE)** is useful for cases where values differ significantly between time points and outliers have a significant impact.

* **Mean Absolute Scaled Error (MASE)** is ideal for datasets that are cyclical in nature or have seasonal properties.

In [7]:
get_accuracy_metrics_response = forecast.get_accuracy_metrics(PredictorArn=predictor_arn)
wql = get_accuracy_metrics_response['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses']
accuracy_scores = get_accuracy_metrics_response['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]

print(f"Weighted Quantile Loss (wQL): {json.dumps(wql, indent=2)}")

print(f"Root Mean Square Error (RMSE): {accuracy_scores['RMSE']}")

print(f"Weighted Absolute Percentage Error (WAPE): {accuracy_scores['WAPE']}")

print(f"Mean Absolute Percentage Error (MAPE): {accuracy_scores['MAPE']}")

print(f"Mean Absolute Scaled Error (MASE): {accuracy_scores['MASE']}")

Weighted Quantile Loss (wQL): [
  {
    "Quantile": 0.9,
    "LossValue": 0.13846153846153844
  },
  {
    "Quantile": 0.5,
    "LossValue": 0.38461538461538464
  },
  {
    "Quantile": 0.1,
    "LossValue": 0.10769230769230768
  }
]
Root Mean Square Error (RMSE): 0.38682272190477407
Weighted Absolute Percentage Error (WAPE): 0.4653846153846153
Mean Absolute Percentage Error (MAPE): 0.051818181818181826
Mean Absolute Scaled Error (MASE): 1e-130


In [68]:
#download backtest exports
#!aws s3 sync s3://forecast-exp-1111/backtest_exports/ ./backtest_exports

In [69]:
#accuracy_metric_values_files = glob.glob(
#                                    os.path.join(os.getcwd(),
#                                                 'backtest_exports', 
#                                                 'accuracy-metrics-values',
#                                                 '*.csv'))

In [70]:
#accuracy_metric_values_files

In [71]:
#accuracy_metric_values = [pd.read_csv(f,sep=',') for f in accuracy_metric_values_files]

In [72]:
#acc_01 = pd.concat(accuracy_metric_values, axis=0).sort_values(by=['item_id','backtestwindow_end_time'])

In [73]:
#acc_01[acc_01['backtest_window']=='Summary']